In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar 

!rm VOCtrainval_06-Nov-2007.tar
!rm VOCtest_06-Nov-2007.tar

VOCdevkit/VOC2007/ has
- <b>Annotations:-</b>It has .xml files that contain the description of the objects in the image. There is one xml file for each image.
- <b>ImageSets</b>
  - <b>Layout</b>
    - test.txt
	- train.txt
	- trainval.txt
	- val.txt
  - <b>Main:-</b>It has the following set of files for each class. These are for object detection task.
    - horse_test.txt
	- horse_train.txt
	- horse_trainval.txt
	- horse_val.txt
	.
	.
	.
	- train.txt
	- test.txt
	- val.txt
	- trainval.txt
  - <b>Segmentation:-</b>It has the following set of files. These are for semantic segmentation task
    - test.txt
    - train.txt
	- trainval.txt
	- val.txt
- <b>JPEGImages:-</b>It has .jpg images
- <b>SegmentationClass:-</b>These have .png images with class segmentation.
- <b>SegmentationObject:-</b>These have .png images with object segmentation.

In [ ]:
import csv
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
import torch
import torchvision 
from tqdm import tqdm
import torch.nn as nn
from torchsummary import summary
from torchvision.transforms import Resize
from torch.utils.data import DataLoader
from PIL import Image

##Setting the device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.autograd.set_detect_anomaly(True)

# Preprocess data

In [ ]:
classes = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 
           'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 
           'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14, 
           'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19}

for grp in ['train','test','val']:

  rows = [['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'cls_id']]
    
  with open(f'VOCdevkit/VOC2007/ImageSets/Main/{grp}.txt', 'r') as file:
    img_ids = [img_id.strip() for img_id in file.read().strip().split('\n')]
    
    for img_id in img_ids:
    
      filename = f'VOCdevkit/VOC2007/JPEGImages/{img_id}.jpg'
      xml = f'VOCdevkit/VOC2007/Annotations/{img_id}.xml'
      tree = ET.parse(xml)
      root = tree.getroot()
    
      for obj in root.findall('object'):
            
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        keys = set(classes.keys())
        if cls in keys and int(difficult) != 1:
          cls_id = classes[cls]
          box = obj.find('bndbox')
          xmin = int(box.find('xmin').text)
          ymin = int(box.find('ymin').text)
          xmax = int(box.find('xmax').text)
          ymax = int(box.find('ymax').text)
          rows.append([filename, xmin, ymin, xmax, ymax, cls_id])
        else:continue
        
  with open(f'VOCdevkit/VOC2007/{grp}.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

In [ ]:
train = pd.read_csv('VOCdevkit/VOC2007/train.csv')
train['cls_id'].value_counts().sort_index()

In [ ]:
val = pd.read_csv('VOCdevkit/VOC2007/val.csv')
val ['cls_id'].value_counts().sort_index()

In [ ]:
test = pd.read_csv('VOCdevkit/VOC2007/test.csv')
test['cls_id'].value_counts().sort_index()

In [ ]:
train.head()

## Prepare train and validation sets

In [ ]:
x_train = []
y_train = []

x_val = []
y_val = []

with open('VOCdevkit/VOC2007/train.csv', 'r') as file:
  train_reader = csv.reader(file)
  next(train_reader)
  for line in train_reader:
    x_train.append(line[0])
    y_train.append([int(value) for value in line[1:]])

with open('VOCdevkit/VOC2007/val.csv', 'r') as file:
  val_reader = csv.reader(file)
  next(val_reader)
  for line in val_reader:
    x_val.append(line[0])
    y_val.append([int(value) for value in line[1:]])

## Custom Dataset Class

In [ ]:
class Dataset(torch.utils.data.Dataset) :

  def __init__(self, images, labels):    
    self.images = images
    self.labels = labels
  
  def __len__(self):
    return len(self.images)
  
  def __getitem__(self, idx):
    img_path = self.images[idx]
    label = self.labels[idx]

    image = torchvision.io.read_image(img_path)
    height, width = image.shape[1:]
    image = torchvision.transforms.ToPILImage()(image)
    image = torchvision.transforms.Resize((448,448))(image)
    image = torchvision.transforms.ToTensor()(image)
    image = image/255
    label_matrix = torch.zeros((30,7,7))
  
    x_min = label[0]
    y_min = label[1]
    x_max = label[2]
    y_max = label[3]
    clss = label[4]
    x = (x_min+x_max)/2/width
    y = (y_min+y_max)/2/height   
    w = (x_max-x_min)/width
    h = (y_max-y_min)/height
    loc = [7*x, 7*y]
    loc_y = int(loc[1])
    loc_x = int(loc[0])
    x = loc[0] - loc_x
    y = loc[1] - loc_y
  
    # note:- these are matrix co-ordinates
    label_matrix[20:25, loc_y, loc_x] = torch.tensor([1, x, y, w, h])
    label_matrix[clss, loc_y, loc_x] = 1

    return image, label_matrix

## Network

### Model

In [ ]:
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(CNNBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
    self.leakyrelu = nn.LeakyReLU(0.1)
  
  def forward(self, x):
    return self.leakyrelu(self.conv(x))

class YoloV1(nn.Module):
  def __init__(self, architecture):
    super(YoloV1, self).__init__()
    self.darknet = self._create_layers(architecture)
      
  def forward(self, x):
    return self.darknet(x)
      
  def _create_layers(self, architecture, in_channels=3):
    layers = []
    #architecture should have two parts. First part should contain conv layers and second part should contain linear layers.
    for x in architecture[0]:
      if type(x) == tuple:
        layers.append(CNNBlock(in_channels, out_channels=x[1], kernel_size=x[0], stride=x[2], padding=x[0]//2))
        in_channels = x[1]
      elif type(x) == str:
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
      elif type(x) == list:
        for y in range(x[-1]):
          for z in x[:-1]:
            layers.append(CNNBlock(in_channels, out_channels=z[1], kernel_size=z[0], stride=z[2], padding=z[0]//2))
            in_channels = z[1]
    
    layers.append(nn.Flatten())
    in_features = architecture[1][0]
    for x in architecture[1][1:]:
      layers.append(nn.Linear(in_features, x))
      in_features = x
      
    return nn.Sequential(*layers)

###Loss

In [ ]:
class YoloLoss(nn.Module):

  def __init__(self, S=7, B=2, C=20):
    super(YoloLoss, self).__init__()
    self.S = S
    self.B = B
    self.C = C
    self.lambda_coord = 0.5
    self.lambda_noobj = 5
  
  #pred and true are (m,30,s,s) tensors
  def forward(self, pred, true):
    pred = torch.reshape(pred,(-1, self.B*5+self.C, self.S, self.S))
    true = torch.reshape(true,(-1, self.B*5+self.C, self.S, self.S))
    iou_b1 = IOU(yolo_to_pixels(pred[:,21:25,:,:]), yolo_to_pixels(true[:,21:25,:,:]))
    iou_b2 = IOU(yolo_to_pixels(pred[:,26:30,:,:]), yolo_to_pixels( true[:,21:25,:,:]))
    ious = torch.cat([iou_b1, iou_b2], dim=1)                         #ious is an (m,2,s,s) tensor
    iou_maxes, best_box = torch.max(ious, dim=1, keepdim=True)        #iou_maxes and best_box are (m,1,s,s) tensors
    exists_box = true[:,20:21,:,:]                                    #exists_box is an (m,1,s,s) tensor
    
    #box loss
    #box_true and box_pred are (m, 4, s, s) tensors
    box_pred = exists_box * (best_box*pred[:,26:30,:,:]+(1-best_box)*pred[:,21:25,:,:])
    box_pred[:,2:,:,:] = torch.sign(box_pred[:,2:,:,:])*torch.sqrt(torch.abs(box_pred[:,2:,:,:]+1e-6))
    box_true = exists_box * true[:,21:25,:,:]
    box_true[:,2:,:,:] = torch.sqrt(box_true[:,2:,:,:])
    diff_box = box_true-box_pred
    box_loss = torch.sum(torch.square(diff_box))
    
    #object loss
    obj_pred = exists_box*(best_box*pred[:,25:26,:,:]+(1-best_box)*pred[:,20:21,:,:])
    obj_true = exists_box * true[:,20:21,:,:]
    diff_obj = obj_true-obj_pred
    obj_loss = torch.sum(torch.square(diff_obj))
    
    #no object loss
    noobj_pred = (1-exists_box) * ((pred[:,25:26,:,:]+pred[:,20:21,:,:])/2)
    noobj_true = (1-exists_box) * true[:,20:21,:,:]
    diff_noobj = noobj_true-noobj_pred
    noobj_loss = torch.sum(torch.square(diff_noobj))
    
    #class loss
    class_pred = exists_box * pred[:,:20,:,:]
    class_true = exists_box * true[:,:20,:,:]
    diff_class = class_true-class_pred
    class_loss = torch.sum(torch.square(diff_class))
    
    #total loss
    loss = self.lambda_coord*box_loss + obj_loss + self.lambda_noobj*noobj_loss + class_loss
    
    return loss

###Helper Functions

In [ ]:
def yolo_to_pixels(boxes, S=7, N=448):
  new_boxes = boxes.clone().detach()
  x_ind_tensor = torch.reshape(torch.arange(0,7),(1,1,1,7)).to(device)
  y_ind_tensor = torch.reshape(torch.arange(0,7),(1,1,7,1)).to(device)
  new_boxes[:,0:1,:,:] = (new_boxes[:,0:1,:,:]!=0)*(x_ind_tensor+new_boxes[:,0:1,:,:])*(N/S)
  new_boxes[:,1:2,:,:] = (new_boxes[:,1:2,:,:]!=0)*(y_ind_tensor+new_boxes[:,1:2,:,:])*(N/S)
  new_boxes[:,2:4,:,:] = new_boxes[:,2:4,:,:]*N
  return new_boxes

def pixels_to_yolo(img_paths, labels):
  yolo_images = []
  yolo_labels = []
  for img_path, label in zip(img_paths, labels):
    image = torchvision.io.read_image(img_path)
    height, width = image.shape[1:]
    image = torchvision.transforms.ToPILImage()(image)
    image = torchvision.transforms.Resize((448,448))(image)
    image = torchvision.transforms.ToTensor()(image)
    image = image/255
    label_matrix = torch.zeros((30,7,7))
    
    x_min = label[0]
    y_min = label[1]
    x_max = label[2]
    y_max = label[3]
    clss = label[4]
    x = (x_min+x_max)/2/width
    y = (y_min+y_max)/2/height   
    w = (x_max-x_min)/width
    h = (y_max-y_min)/height
    loc = [7*x, 7*y]
    loc_y = int(loc[1])
    loc_x = int(loc[0])
    x = loc[0] - loc_x
    y = loc[1] - loc_y
    
    # note:- these are matrix co-ordinates
    label_matrix[ 20:25, loc_y, loc_x] = torch.tensor([1, x, y, w, h])
    label_matrix[ clss, loc_y, loc_x] = 1
    
    yolo_images.append(image)
    yolo_labels.append(label_matrix)
  
  return torch.stack(yolo_images), torch.stack(yolo_labels)
#pred, true are of shape (m,4,s,s) with format [x_mid, y_mid, width, height]
def IOU(pred, true, S=7):
  #following are of shape (m,1,s,s)
  x_pred = pred[:,0:1,:,:]
  y_pred = pred[:,1:2,:,:]
  w_pred = pred[:,2:3,:,:]
  h_pred = pred[:,3:4,:,:]
  x_true = true[:,0:1,:,:]
  y_true = true[:,1:2,:,:]
  w_true = true[:,2:3,:,:]
  h_true = true[:,3:4,:,:]
  x_max = torch.max(x_pred-w_pred/2, x_true-w_true/2)
  y_max = torch.max(y_pred-h_pred/2, y_true-h_true/2)
  x_min = torch.min(x_pred+w_pred/2, x_true+w_true/2)
  y_min = torch.min(y_pred+h_pred/2, y_true+h_true/2)
  
  intersection = (x_min-x_max) * (y_min-y_max)
  union = w_pred * h_pred + w_true * h_true - intersection
  
  iou = intersection/union
  iou[iou!=iou] = 0

  return iou

def yolo_to_boxes(out, S=7, box_thre=0.9):
  #out is of shape (m,30,7,7)
  #boxes1, boxes2, boxes are of shape (m,4,7,7)
  boxes1 = out[:,21:25,:,:]
  boxes2 = out[:,26:30,:,:]
  prob_scores = torch.cat([out[:,20:21,:,:],out[:,25:26,:,:]],dim=1) #prob_scores is of shape (m,2,7,7)
  best_prob, best_box = torch.max(prob_scores, dim=1, keepdim=True)  #best_prob, best_box are of shape (m,1,7,7)
  boxes = boxes1*(1-best_box)+best_box*boxes2
  #clss is of shape (m,1,7,7)
  clss = torch.argmax(out[:,:20,:,:],dim=1,keepdim=True)
  bboxes = torch.cat([clss, best_prob, yolo_to_pixels(boxes)], dim=1)#bboxes is of shape (m,6,7,7)
  bboxes = torch.reshape(bboxes, (-1, 6, S*S))
  new_bboxes = {}
  
  for i in range(bboxes.shape[0]):
    new_bboxes[i]=[]
    for cell in range(bboxes.shape[-1]):
      temp=[]
      if bboxes[i,:,cell][1]>box_thre:
        for value in bboxes[i,:,cell]:
          temp.append(round(float(value),4))
        new_bboxes[i].append(temp)

  return new_bboxes

def iou(pred, true, S=7):
  #following are of shape (m,1,s,s)
  x_pred = pred[0]
  y_pred = pred[1]
  w_pred = pred[2]
  h_pred = pred[3]
  x_true = true[0]
  y_true = true[1]
  w_true = true[2]
  h_true = true[3]
  x_max = torch.max(x_pred-w_pred/2, x_true-w_true/2)
  y_max = torch.max(y_pred-h_pred/2, y_true-h_true/2)
  x_min = torch.min(x_pred+w_pred/2, x_true+w_true/2)
  y_min = torch.min(y_pred+h_pred/2, y_true+h_true/2)
  
  intersection = (x_min-x_max) * (y_min-y_max)
  union = w_pred * h_pred + w_true * h_true - intersection
  
  iou = intersection/union
  iou[iou!=iou] = 0

  return iou

#pred is of format {0:[[class, box_prob, x_mid, y_mid, width, height],...],...}
def NMS(pred, iou_thre=0.5):
  for key in pred:
    pred[key] = sorted(pred[key], key=lambda x:x[1], reverse=True)
  nms_boxes = {}
  
  for key in pred:
    nms_boxes[key] = []
    while pred[key]:
      chosen_box = pred[key].pop(0)
      pred[key] = [box 
                   for box in pred[key] 
                   if box[0]!=chosen_box[0] or
                   iou(torch.tensor(box),torch.tensor(chosen_box))<iou_thre]
      nms_boxes[key].append(chosen_box)

  return nms_boxes

def plot_image(boxes, img_path):
  size = 448
  image = cv.imread(img_path)
  image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image = cv.resize(image,(448,448))
  for box in boxes:
    cls = int(box[0])
    x = box[2]
    y = box[3]
    w = box[4]
    h = box[5]
    x_min = int(max(x-w/2,0))
    y_min = int(max(y-h/2,10))
    x_max = int(min(x+w/2,448))
    y_max = int(min(y+h/2,448))
    cv.rectangle(image,(x_min,y_min),(x_max,y_max),(255,255,255),2)
    cv.putText(image, list(classes.keys())[list(classes.values()).index(cls)], \
            (x_min, y_min-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    plt.imshow(image) 

def MAP(pred, true, n_classes, iou_thresh=0.5):
  #pred, true are of format [idx, class, obj_prob, x1, y1, x2, y2]
  avg_prec = []
    
  for c in range(n_classes):
    predictions = []
    truths = []
        
    for prediction in pred:
      if prediction[1]==c:
        predictions.append(prediction)
        
      for truth in true:
        if truth[1]==c:
          truths.append(truth)
        
    boxes_per_img = Counter([truth[0] for truth in truths])
        
    for key, val in boxes_per_img.items():
      boxes_per_img[key] = torch.zeros(val)
            
      predictions.sort(key=lambda x:x[2], reverse=True)
      TP = torch.zeros(len(predictions))
      FP = torch.zeros(len(predictions))
      n_truths  = len(truths)
        
      for pred_idx, prediction in enumerate(predictions):
        truth_img = [truth for truth in truths if truth[0]==prediction[0]]
        n_truths_img = len(truth_img)
        best_iou = 0
            
        for idx, truth in enumerate(truth_img):
          iou = IOU(prediction[3:], truth[3:])
          if iou>best_iou:
            best_iou = iou
            best_truth_idx = idx
            
          if best_iou>iou_thresh:
            if boxes_per_img[prediction[0]][best_truth_idx]==0:
              TP[pred_idx]=1
              boxes_per_img[prediction[0]][best_truth_idx]=1
            else:
              FP[pred_idx]=1
          else:
            FP[pred_idx]=1
            
      TP_cumsum = torch.cumsum(TP, dim=0)
      FP_cumsum = torch.cumsum(FP, dim=0)
      recall = TP_cumsum/n_truths
      precision = TP_cumsum/(TP_cumsum+FP_cumsum)
      precision = torch.cat(torch.tensor([1]), precision)
      recall = torch.cat(torch.tensor([0]), recall)
      avg_prec.append(torch.trapz(precision,recall))
        
    return sum(avg_prec)/len(avg_prec)

def save_chkpt(model, optim, filename='drive/MyDrive/chkpt.pth.tar'):
  chkpt = {'state_dict':model.state_dict(),'optimizer':optim.state_dict()}
  torch.save(chkpt, filename)

def load_chkpt(model, optim, filename='drive/MyDrive/chkpt.pth.tar'):
  chkpt = torch.load(filename)
  model.load_state_dict(chkpt['state_dict'])
  optim.load_state_dict(chkpt['optimizer'])

##Training

###Architecture

In [ ]:
C = 20
S = 7
B = 2
N = 448

architecture = [[
#tuple : (kernel_size, n_filters, stride)
(7,64,2),
#M : maxpooling layer with kernel_size=2 and stride=2
'M',
(3,192,1),
'M',
(1,128,1),
(3,256,1),
(1,256,1),
(3,512,1),
'M',
#list : [tuples, repeat]
[(1,256,1),(3,512,1),4],
(1,512,1),
(3,1024,1),
'M',
[(1,512,1),(3,1024,1),2],
(3,1024,1),
(3,1024,2),
(3,1024,1),
(3,1024,1)],
[
# int : n_neurons
7*7*1024,
4096,
S*S*(C+B*5)
]]

In [ ]:
load_model = True
model = YoloV1(architecture).to(device)
yololoss = YoloLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=2e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

if load_model:
  load_chkpt(model, optimizer)

In [ ]:
summary(model,(3,448,448))

In [ ]:
epoch_losses = []
def fit(x_train, y_train, model, optimizer, epochs):
  train_set = Dataset(x_train, y_train)
  train_loader = DataLoader(train_set, 16, True)
  for epoch in range(epochs):
    batch_losses = []
    if epoch%1==0 and epoch!=0:
      save_chkpt(model, optimizer)
    loop = tqdm(train_loader,  position=0, leave=True)
    for x, y in loop:
      x, y = x.to(device), y.to(device)
      y_hat = model(x)
      loss = yololoss(y_hat, y)
      batch_losses.append(loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loop.set_postfix(loss=loss)

    epoch_losses.append(sum(batch_losses)/len(batch_losses))
  return

In [ ]:
def predict(x_test, model):
  images = []
  for img_path in x_test:
    image = torchvision.io.read_image(img_path)
    height, width = image.shape[1:]
    image = torchvision.transforms.ToPILImage()(image)
    image = torchvision.transforms.Resize((448,448))(image)
    image = torchvision.transforms.ToTensor()(image)
    image = image/255
    images.append(image)
  
  y_hat = model(torch.stack(images))
  y_hat = torch.reshape(y_hat,(len(x_test),30,7,7))
 
  boxes = NMS(yolo_to_boxes(y_hat))
  return boxes

In [ ]:
fit(x_train,y_train,model,optimizer,10)

In [ ]:
index = 0
x_test = ['VOCdevkit/VOC2007/JPEGImages/000017.jpg']
boxes = predict(x_test,model)
plot_image(boxes[0],x_test[0])